In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [39]:
captions = []
ground_truth = []
predicted = []
confidence_scores = []
inference_time_list = []

result_dir = '/Users/twel/projects/DKH-Thesis/src/result'
result_json_list = os.listdir(result_dir)
for item in result_json_list:
    result_json_dir = os.path.join(result_dir, item)
    with open(result_json_dir, 'r') as f:
        result_json = json.load(f)
        captions.append(result_json['caption'])
        ground_truth.append(result_json['ground_truth'])
        predicted.append(1 if result_json['final_result']['OOC'] else 0)
        confidence_scores.append(result_json['final_result']['confidence_score'])
        inference_time_list.append(result_json['inference_time'])

In [40]:
print(len(captions))
print(len(ground_truth))
print(len(predicted))
print(len(inference_time_list))

876
876
876
876


In [41]:
df = pd.DataFrame({
    'caption': captions,
    'ground_truth': ground_truth,
    'predicted': predicted,
    'confidence_score': confidence_scores,
    'inference_time': inference_time_list
})
df['adjusted_predicted'] = df.apply(
    lambda row: row['predicted'] if row['confidence_score'] >= 8 else 1 - row['predicted'], axis=1
)
adjusted_predicted = df['adjusted_predicted'].tolist()
df

,caption,ground_truth,predicted,confidence_score,inference_time,adjusted_predicted
0,Keith Potter netted the purple lobster on Tues...,1,1,10,7.069135,1
1,A photograph shows a soldier carrying a donkey...,1,1,8,4.467112,1
2,Trucks driving to the British port of Dover in...,0,1,10,4.804000,1
3,A Ghana policeman walks with his dog during a ...,0,1,8,12.300732,1
4,the methane bubbles that form naturally and ar...,0,1,10,5.936312,1
...,...,...,...,...,...,...
871,"Also on Friday, vendors sell roses of all colo...",0,1,8,7.431300,1
872,A report from the Centers for Disease Control ...,1,1,8,8.137317,1
873,A photograph shows buses purchased by Black Li...,1,1,8,3.710260,1
874,"A video showing a massive, dangerous waterspou...",1,0,10,20.681890,0


In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

class_names = ["NOOC", "OOC"]

# Generate Classification Report
report = classification_report(ground_truth, predicted, target_names=class_names)
print("\nClassification Report:")
print(report)
print("################################")

# calculate per-class accuracy
cm = confusion_matrix(ground_truth, predicted)
class_accuracies = cm.diagonal() / cm.sum(axis=1)
print("Per-Class Accuracy:")
for class_name, acc in zip(class_names, class_accuracies):
    print(f"{class_name}: {acc:.4f}")
print("################################")

# Calculate average inference time
average_time = sum(inference_time_list) / len(inference_time_list)
print(f"Average Inference Time: {average_time:.6f} seconds")


Classification Report:
              precision    recall  f1-score   support

        NOOC       0.82      0.22      0.35       437
         OOC       0.55      0.95      0.70       439

    accuracy                           0.59       876
   macro avg       0.69      0.59      0.53       876
weighted avg       0.69      0.59      0.53       876

################################
Per-Class Accuracy:
NOOC: 0.2243
OOC: 0.9522
################################
Average Inference Time: 10.466173 seconds


In [45]:
cm

array([[ 98, 339],
       [ 21, 418]])